Tools

Tools let agents take actions: things like fetching data, running code, calling external APIs, and even using a computer. There are three classes of tools in the Agent SDK:

* Hosted tools: these run on LLM servers alongside the AI models. OpenAI offers retrieval, web search and computer use as 
  hosted tools.

* Function calling: these allow you to use any Python function as a tool.

* Agents as tools: this allows you to use an agent as a tool, allowing Agents to call other agents without handing off to them.

Hosted tools

OpenAI offers a few built-in tools when using the OpenAIResponsesModel:

* The WebSearchTool lets an agent search the web.

* The FileSearchTool allows retrieving information from your OpenAI Vector Stores.

* The ComputerTool allows automating computer use tasks.

* The CodeInterpreterTool lets the LLM execute code in a sandboxed environment.

* The HostedMCPTool exposes a remote MCP server's tools to the model.

* The ImageGenerationTool generates images from a prompt.

* The LocalShellTool runs shell commands on your machine.

اوزار

ٹولز ایجنٹوں کو کارروائیاں کرنے دیتے ہیں: ڈیٹا حاصل کرنا، کوڈ چلانا، بیرونی APIs کو کال کرنا، اور یہاں تک کہ کمپیوٹر کا استعمال کرنا۔ ایجنٹ SDK میں ٹولز کی تین کلاسیں ہیں:

* ہوسٹڈ ٹولز: یہ AI ماڈلز کے ساتھ LLM سرورز پر چلتے ہیں۔ OpenAI بازیافت، ویب تلاش اور کمپیوٹر کے استعمال کی پیشکش کرتا ہے۔ 
میزبانی کے اوزار.

* فنکشن کالنگ: یہ آپ کو کسی بھی Python فنکشن کو بطور ٹول استعمال کرنے کی اجازت دیتا ہے۔

* ایجنٹوں کو بطور اوزار: یہ آپ کو ایجنٹ کو بطور ٹول استعمال کرنے کی اجازت دیتا ہے، جس سے ایجنٹوں کو دوسرے ایجنٹوں کو ان کے حوالے کیے بغیر کال کرنے کی اجازت ملتی ہے۔

میزبانی کے اوزار

OpenAIResponsesModel کا استعمال کرتے وقت OpenAI چند بلٹ ان ٹولز پیش کرتا ہے:

* WebSearchTool ایک ایجنٹ کو ویب پر تلاش کرنے دیتا ہے۔

* فائل سرچ ٹول آپ کے اوپن اے آئی ویکٹر اسٹورز سے معلومات حاصل کرنے کی اجازت دیتا ہے۔

* ComputerTool کمپیوٹر کے استعمال کے کاموں کو خودکار کرنے کی اجازت دیتا ہے۔

* CodeInterpreterTool LLM کو سینڈ باکس والے ماحول میں کوڈ پر عمل درآمد کرنے دیتا ہے۔

* ہوسٹڈ ایم سی پی ٹول ریموٹ ایم سی پی سرور کے ٹولز کو ماڈل کے سامنے لاتا ہے۔

* امیج جنریشن ٹول پرامپٹ سے تصاویر تیار کرتا ہے۔

* LocalShellTool آپ کی مشین پر شیل کمانڈ چلاتا ہے۔

In [ ]:
from agents import Agent, FileSearchTool, Runner, WebSearchTool

agent = Agent(
    name="Assistant",
    tools=[
        WebSearchTool(),
        FileSearchTool(
            max_num_results=3,
            vector_store_ids=["VECTOR_STORE_ID"],
        ),
    ],
)

async def main():
    result = await Runner.run(agent, "Which coffee shop should I go to, taking into account my preferences and the weather today in SF?")
    print(result.final_output)

Function tools

You can use any Python function as a tool. The Agents SDK will setup the tool automatically:

* The name of the tool will be the name of the Python function (or you can provide a name)

* Tool description will be taken from the docstring of the function (or you can provide a description)

* The schema for the function inputs is automatically created from the function's arguments

* Descriptions for each input are taken from the docstring of the function, unless disabled

We use Python's inspect module to extract the function signature, along with griffe to parse docstrings and pydantic for schema creation.

فنکشن ٹولز

آپ کسی بھی Python فنکشن کو بطور ٹول استعمال کر سکتے ہیں۔ ایجنٹ SDK ٹول کو خود بخود سیٹ اپ کر دے گا:

* ٹول کا نام Python فنکشن کا نام ہوگا (یا آپ ایک نام دے سکتے ہیں)

* ٹول کی تفصیل فنکشن کی دستاویز سے لی جائے گی (یا آپ تفصیل فراہم کر سکتے ہیں)

* فنکشن ان پٹ کے لیے اسکیما فنکشن کے دلائل سے خود بخود بن جاتا ہے۔

* ہر ان پٹ کی تفصیل فنکشن کی دستاویز سے لی جاتی ہے، جب تک کہ غیر فعال نہ ہو۔

ہم فنکشن کے دستخط نکالنے کے لیے Python کے انسپیکٹ ماڈیول کا استعمال کرتے ہیں، ساتھ ہی griffe کو docstrings کو پارس کرنے کے لیے اور pydantic کو سکیما بنانے کے لیے استعمال کرتے ہیں۔

In [ ]:
import json

from typing_extensions import TypedDict, Any

from agents import Agent, FunctionTool, RunContextWrapper, function_tool


class Location(TypedDict):
    lat: float
    long: float

@function_tool  
async def fetch_weather(location: Location) -> str:
    
    """Fetch the weather for a given location.

    Args:
        location: The location to fetch the weather for.
    """
    # In real life, we'd fetch the weather from a weather API
    return "sunny"


@function_tool(name_override="fetch_data")  
def read_file(ctx: RunContextWrapper[Any], path: str, directory: str | None = None) -> str:
    """Read the contents of a file.

    Args:
        path: The path to the file to read.
        directory: The directory to read the file from.
    """
    # In real life, we'd read the file from the file system
    return "<file contents>"


agent = Agent(
    name="Assistant",
    tools=[fetch_weather, read_file],  
)

for tool in agent.tools:
    if isinstance(tool, FunctionTool):
        print(tool.name)
        print(tool.description)
        print(json.dumps(tool.params_json_schema, indent=2))
        print()

Custom function tools

Sometimes, you don't want to use a Python function as a tool. You can directly create a FunctionTool if you prefer. You'll need to provide:

* name

* description

* params_json_schema, which is the JSON schema for the arguments
* on_invoke_tool, which is an async function that receives a ToolContext and the arguments as a JSON string, and must return 
  the tool output as a string.

حسب ضرورت فنکشن ٹولز

کبھی کبھی، آپ Python فنکشن کو بطور ٹول استعمال نہیں کرنا چاہتے۔ اگر آپ چاہیں تو آپ براہ راست فنکشن ٹول بنا سکتے ہیں۔ آپ کو فراہم کرنے کی ضرورت ہوگی:

*نام

*تفصیل

* params_json_schema، جو کہ دلائل کے لیے JSON اسکیما ہے۔
* on_invoke_tool، جو ایک async فنکشن ہے جو ToolContext اور آرگیومنٹس کو JSON سٹرنگ کے طور پر حاصل کرتا ہے، اور اسے واپس آنا چاہیے۔ 
ٹول آؤٹ پٹ بطور تار۔

In [ ]:
from typing import Any

from pydantic import BaseModel

from agents import RunContextWrapper, FunctionTool

def do_some_work(data: str) -> str:
    return "done"

class FunctionArgs(BaseModel):
    username: str
    age: int

async def run_function(ctx: RunContextWrapper[Any], args: str) -> str:
    parsed = FunctionArgs.model_validate_json(args)
    return do_some_work(data=f"{parsed.username} is {parsed.age} years old")

tool = FunctionTool(
    name="process_user",
    description="Processes extracted user data",
    params_json_schema=FunctionArgs.model_json_schema(),
    on_invoke_tool=run_function,
)

Automatic argument and docstring parsing

As mentioned before, we automatically parse the function signature to extract the schema for the tool, and we parse the docstring to extract descriptions for the tool and for individual arguments. Some notes on that:

1. The signature parsing is done via the inspect module. We use type annotations to understand the types for the arguments, 
    and dynamically build a Pydantic model to represent the overall schema. It supports most types, including Python primitives, Pydantic models, TypedDicts, and more.

2. We use griffe to parse docstrings. Supported docstring formats are google, sphinx and numpy. We attempt to automatically 
   detect the docstring format, but this is best-effort and you can explicitly set it when calling function_tool. You can also disable docstring parsing by setting use_docstring_info to False.

The code for the schema extraction lives in agents.function_schema.

Agents as tools

In some workflows, you may want a central agent to orchestrate a network of specialized agents, instead of handing off control. You can do this by modeling agents as tools.

خودکار دلیل اور docstring پارسنگ

جیسا کہ پہلے ذکر کیا گیا ہے، ہم ٹول کے لیے اسکیما کو نکالنے کے لیے فنکشن کے دستخط کو خود بخود پارس کرتے ہیں، اور ہم ٹول اور انفرادی دلائل کے لیے تفصیل نکالنے کے لیے docstring کو پارس کرتے ہیں۔ اس پر کچھ نوٹ:

1. دستخط کی تجزیہ معائنہ ماڈیول کے ذریعے کی جاتی ہے۔ ہم دلائل کی اقسام کو سمجھنے کے لیے ٹائپ تشریحات کا استعمال کرتے ہیں، 
اور مجموعی اسکیما کی نمائندگی کے لیے متحرک طور پر ایک Pydantic ماڈل بنائیں۔ یہ زیادہ تر اقسام کو سپورٹ کرتا ہے، بشمول Python primitives، Pydantic models، TypedDicts، اور بہت کچھ۔

2. ہم docstrings کو پارس کرنے کے لیے griffe کا استعمال کرتے ہیں۔ معاون ڈاکسٹرنگ فارمیٹس گوگل، اسفنکس اور نمپی ہیں۔ ہم خود بخود کوشش کرتے ہیں۔ 
docstring فارمیٹ کا پتہ لگائیں، لیکن یہ بہترین کوشش ہے اور function_tool کو کال کرتے وقت آپ اسے واضح طور پر سیٹ کر سکتے ہیں۔ آپ use_docstring_info کو False پر سیٹ کر کے docstring parsing کو بھی غیر فعال کر سکتے ہیں۔

اسکیما نکالنے کا کوڈ agents.function_schema میں رہتا ہے۔

ایجنٹوں کو بطور اوزار

کچھ ورک فلو میں، آپ کسی مرکزی ایجنٹ سے کنٹرول ختم کرنے کے بجائے خصوصی ایجنٹوں کے نیٹ ورک کو ترتیب دینے کے لیے چاہیں گے۔ آپ ماڈلنگ ایجنٹوں کو بطور ٹولز کر سکتے ہیں۔

In [ ]:
from agents import Agent, Runner
import asyncio

spanish_agent = Agent(
    name="Spanish agent",
    instructions="You translate the user's message to Spanish",
)

french_agent = Agent(
    name="French agent",
    instructions="You translate the user's message to French",
)

orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions=(
        "You are a translation agent. You use the tools given to you to translate."
        "If asked for multiple translations, you call the relevant tools."
    ),
    tools=[
        spanish_agent.as_tool(
            tool_name="translate_to_spanish",
            tool_description="Translate the user's message to Spanish",
        ),
        french_agent.as_tool(
            tool_name="translate_to_french",
            tool_description="Translate the user's message to French",
        ),
    ],
)

async def main():
    result = await Runner.run(orchestrator_agent, input="Say 'Hello, how are you?' in Spanish.")
    print(result.final_output)

Customizing tool-agents

The agent.as_tool function is a convenience method to make it easy to turn an agent into a tool. It doesn't support all configuration though; for example, you can't set max_turns. For advanced use cases, use Runner.run directly in your tool implementation:

ٹول ایجنٹوں کو حسب ضرورت بنانا

agent.as_tool فنکشن ایک سہولت کا طریقہ ہے جس سے ایجنٹ کو ٹول میں تبدیل کرنا آسان ہو جاتا ہے۔ اگرچہ یہ تمام ترتیب کی حمایت نہیں کرتا؛ مثال کے طور پر، آپ max_turns سیٹ نہیں کر سکتے۔ اعلی درجے کے استعمال کے معاملات کے لیے، Runner.run کو براہ راست اپنے ٹول کے نفاذ میں استعمال کریں:

In [ ]:
@function_tool
async def run_my_agent() -> str:
    """A tool that runs the agent with custom configs"""

    agent = Agent(name="My agent", instructions="...")

    result = await Runner.run(
        agent,
        input="...",
        max_turns=5,
        run_config=...
    )

    return str(result.final_output)

Custom output extraction

In certain cases, you might want to modify the output of the tool-agents before returning it to the central agent. This may be useful if you want to:

* Extract a specific piece of information (e.g., a JSON payload) from the sub-agent's chat history.

* Convert or reformat the agent’s final answer (e.g., transform Markdown into plain text or CSV).

* Validate the output or provide a fallback value when the agent’s response is missing or malformed.

You can do this by supplying the custom_output_extractor argument to the as_tool method:

اپنی مرضی کے مطابق آؤٹ پٹ نکالنا

بعض صورتوں میں، آپ مرکزی ایجنٹ کو واپس کرنے سے پہلے ٹول ایجنٹ کے آؤٹ پٹ میں ترمیم کرنا چاہیں گے۔ اگر آپ چاہیں تو یہ مفید ہو سکتا ہے:

* ذیلی ایجنٹ کی چیٹ ہسٹری سے معلومات کا ایک مخصوص حصہ (مثال کے طور پر، JSON پے لوڈ) نکالیں۔

* ایجنٹ کے حتمی جواب کو تبدیل یا دوبارہ فارمیٹ کریں (مثال کے طور پر، مارک ڈاؤن کو سادہ متن یا CSV میں تبدیل کریں)۔

* جب ایجنٹ کا جواب غائب یا خراب ہو تو آؤٹ پٹ کی توثیق کریں یا فال بیک ویلیو فراہم کریں۔

آپ as_tool طریقہ کو custom_output_extractor دلیل فراہم کرکے ایسا کرسکتے ہیں:

In [ ]:
async def extract_json_payload(run_result: RunResult) -> str:
    # Scan the agent’s outputs in reverse order until we find a JSON-like message from a tool call.
    for item in reversed(run_result.new_items):
        if isinstance(item, ToolCallOutputItem) and item.output.strip().startswith("{"):
            return item.output.strip()
    # Fallback to an empty JSON object if nothing was found
    return "{}"


json_tool = data_agent.as_tool(
    tool_name="get_data_json",
    tool_description="Run the data agent and return only its JSON payload",
    custom_output_extractor=extract_json_payload,
)

Conditional tool enabling

You can conditionally enable or disable agent tools at runtime using the is_enabled parameter. This allows you to dynamically filter which tools are available to the LLM based on context, user preferences, or runtime conditions.

مشروط ٹول کو فعال کرنا

آپ is_enabled پیرامیٹر کا استعمال کرتے ہوئے رن ٹائم پر ایجنٹ ٹولز کو مشروط طور پر فعال یا غیر فعال کر سکتے ہیں۔ یہ آپ کو متحرک طور پر فلٹر کرنے کی اجازت دیتا ہے کہ کون سے ٹولز سیاق و سباق، صارف کی ترجیحات، یا رن ٹائم حالات کی بنیاد پر LLM کے لیے دستیاب ہیں۔

In [ ]:
import asyncio
from agents import Agent, AgentBase, Runner, RunContextWrapper
from pydantic import BaseModel

class LanguageContext(BaseModel):
    language_preference: str = "french_spanish"

def french_enabled(ctx: RunContextWrapper[LanguageContext], agent: AgentBase) -> bool:
    """Enable French for French+Spanish preference."""
    return ctx.context.language_preference == "french_spanish"

# Create specialized agents
spanish_agent = Agent(
    name="spanish_agent",
    instructions="You respond in Spanish. Always reply to the user's question in Spanish.",
)

french_agent = Agent(
    name="french_agent",
    instructions="You respond in French. Always reply to the user's question in French.",
)

# Create orchestrator with conditional tools
orchestrator = Agent(
    name="orchestrator",
    instructions=(
        "You are a multilingual assistant. You use the tools given to you to respond to users. "
        "You must call ALL available tools to provide responses in different languages. "
        "You never respond in languages yourself, you always use the provided tools."
    ),
    tools=[
        spanish_agent.as_tool(
            tool_name="respond_spanish",
            tool_description="Respond to the user's question in Spanish",
            is_enabled=True,  # Always enabled
        ),
        french_agent.as_tool(
            tool_name="respond_french",
            tool_description="Respond to the user's question in French",
            is_enabled=french_enabled,
        ),
    ],
)

async def main():
    context = RunContextWrapper(LanguageContext(language_preference="french_spanish"))
    result = await Runner.run(orchestrator, "How are you?", context=context.context)
    print(result.final_output)

asyncio.run(main())

The is_enabled parameter accepts: - Boolean values: True (always enabled) or False (always disabled) - Callable functions: Functions that take (context, agent) and return a boolean - Async functions: Async functions for complex conditional logic

Disabled tools are completely hidden from the LLM at runtime, making this useful for: - Feature gating based on user permissions - Environment-specific tool availability (dev vs prod) - A/B testing different tool configurations - Dynamic tool filtering based on runtime state

Handling errors in function tools

When you create a function tool via @function_tool, you can pass a failure_error_function. This is a function that provides an error response to the LLM in case the tool call crashes.

* By default (i.e. if you don't pass anything), it runs a default_tool_error_function which tells the LLM an error occurred.

* If you pass your own error function, it runs that instead, and sends the response to the LLM.

* If you explicitly pass None, then any tool call errors will be re-raised for you to handle. This could be a 
  ModelBehaviorError if the model produced invalid JSON, or a UserError if your code crashed, etc.

is_enabled پیرامیٹر قبول کرتا ہے: - بولین اقدار: درست (ہمیشہ فعال) یا غلط (ہمیشہ غیر فعال) - کال قابل فنکشنز: فنکشنز جو (سیاق و سباق، ایجنٹ) لیتے ہیں اور بولین واپس کرتے ہیں - Async افعال: پیچیدہ مشروط منطق کے لیے Async افعال

غیر فعال ٹولز رن ٹائم کے وقت LLM سے مکمل طور پر پوشیدہ رہتے ہیں، جو اس کے لیے مفید ہیں: - صارف کی اجازتوں پر مبنی فیچر گیٹنگ - ماحولیات کے لیے مخصوص ٹول کی دستیابی (devs vs prod) - A/B مختلف ٹول کنفیگریشنز کی جانچ کرنا - رن ٹائم حالت کی بنیاد پر متحرک ٹول فلٹرنگ

فنکشن ٹولز میں غلطیوں کو ہینڈل کرنا

جب آپ @function_tool کے ذریعے فنکشن ٹول بناتے ہیں تو آپ ایک ناکامی_error_function پاس کر سکتے ہیں۔ یہ ایک ایسا فنکشن ہے جو ٹول کال کریش ہونے کی صورت میں ایل ایل ایم کو غلطی کا جواب فراہم کرتا ہے۔

* بطور ڈیفالٹ (یعنی اگر آپ کچھ بھی پاس نہیں کرتے ہیں)، یہ ایک ڈیفالٹ_ٹول_ایرر_فنکشن چلاتا ہے جو ایل ایل ایم کو بتاتا ہے کہ کوئی خرابی واقع ہوئی ہے۔

* اگر آپ اپنا ایرر فنکشن پاس کرتے ہیں، تو یہ اس کے بجائے چلتا ہے، اور جواب LLM کو بھیجتا ہے۔

* اگر آپ واضح طور پر کوئی نہیں پاس کرتے ہیں، تو کسی بھی ٹول کال کی خرابیوں کو آپ کے ہینڈل کرنے کے لیے دوبارہ اٹھایا جائے گا۔ یہ ایک ہو سکتا ہے 

ModelBehaviorError اگر ماڈل نے غلط JSON پیدا کیا ہو، یا اگر آپ کا کوڈ کریش ہو جائے تو صارف کی خرابی وغیرہ۔

In [ ]:
from agents import function_tool, RunContextWrapper
from typing import Any

def my_custom_error_function(context: RunContextWrapper[Any], error: Exception) -> str:
    """A custom function to provide a user-friendly error message."""
    print(f"A tool call failed with the following error: {error}")
    return "An internal server error occurred. Please try again later."

@function_tool(failure_error_function=my_custom_error_function)
def get_user_profile(user_id: str) -> str:
    """Fetches a user profile from a mock API.
     This function demonstrates a 'flaky' or failing API call.
    """
    if user_id == "user_123":
        return "User profile for user_123 successfully retrieved."
    else:
        raise ValueError(f"Could not retrieve profile for user_id: {user_id}. API returned an error.")

If you are manually creating a FunctionTool object, then you must handle errors inside the on_invoke_tool function.

اگر آپ دستی طور پر فنکشن ٹول آبجیکٹ بنا رہے ہیں، تو آپ کو on_invoke_tool فنکشن کے اندر موجود غلطیوں کو ہینڈل کرنا ہوگا۔